<a href="https://colab.research.google.com/github/JonNData/naive_bayes/blob/master/notebooks/Naive_Bayes_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np

## Multinomial Naive Bayes
Input X: array of messages  
Input y: array of labels

In [ ]:
class MNNaiveBayes:
  def __init__(self, k=0.5):
    self.k = k
    self.word_probs = []

  def tokenize(self, document):
    """
    Take in a document and return a list of words
    """
    doc = document.lower()
    # remove non-alpha characters
    stop_chars = '''0123456789!()-[]{};:'"\,<>./?@#$%^&*_~'''
    
    tokens = ""
    # iterate through and make each token
    for char in doc:
      if char not in stop_chars:
        tokens += char

    return tokens.split() # now a list of tokens
  
  def count_words(self, X, y):
    """
    X is an array of documents
    y is an array of targets, 0 or 1
    Output a dictionary of {word: (cat0_count, cat1_count)...}
    """
    counts = {}
    for document in X:
      for category in y:
        for token in self.tokenize(document):
          # Initialize a dict entry with 0 counts
          if token not in counts:
            counts[token] = [0,0]
          # Now that it exists, add to the category count for that word
          counts[token][category] += 1
    return counts

  def prior_prob(self, counts):
    
    # Iterate through counts dict and add up each word count by category
    cat0_word_count = cat1_word_count = 0
    for word, (cat0_count, cat1_count) in counts.items():
        cat0_word_count += cat0_count
        cat1_word_count += cat1_count
    
    # Get the prior prob by dividing words in each cat by total words
    cat_0_prior = cat0_word_count / (cat0_word_count + cat1_word_count)
    cat_1_prior = cat1_word_count / (cat0_word_count + cat1_word_count)
    return cat_0_prior, cat_1_prior

  def word_probabilities()
    """
    Output: 





